In [1]:
include("../src/RadialBasisFunction.jl")
include("../src/DiscreteAdjoint.jl")
include("../src/GradientDescentTools.jl")

using LinearAlgebra
using .RadialBasisFunction
using .DiscreteAdjoint
using .GradientDescentTools

## Adjoint method applied to RBF approximation

Fixed some paramters, we obtain an approximated field solving the steady-state heat equation

In [2]:
a, b = 0, 2π
N = 40
X = range(a,b, length=N)
boundary_idx = [1, N]
X_boundary = X[boundary_idx]
internal_idx = 2:(N-1)
X_internal = X[internal_idx]


u(x) = sin.(x)   # Not known in advance
q(x) = -sin.(x)  # Known in advance (param)

boundary_conditions = u(X_boundary)


φ(r) = r .^ 3  # r = |x-x_i| ≥ 0
ddφ(r) = 6*r   #second derivative d²/dx² (because the differential problem deals with u" = d²u/dx²)


n_points_in_stencil = 7


C = global_matrix(X, φ, ddφ, n_points_in_stencil)
u_approximated = solve_RBF(C, X, internal_idx, boundary_idx, boundary_conditions, q)

38-element Vector{Float64}:
  0.16230630440607596
  0.3184441606794323
  0.46639819882924805
  0.6024453236002578
  0.7229552007955176
  0.8247685909891668
  0.9052437503888441
  0.9622961535265828
  0.9944469507504687
  1.000861661907953
  ⋮
 -0.9944469507504649
 -0.96229615352658
 -0.9052437503888425
 -0.8247685909891656
 -0.7229552007955165
 -0.602445323600257
 -0.46639819882924743
 -0.31844416067943265
 -0.16230630440607627

Now we want to find a specific field, $u_{opt}$, that satisfy some properties defined by $J(u)$. $J$ can be thought as a "quality" function if it has to be maximized or as a "cost" if it has to be minimized.  
To reach our goal we can modify some parameters of the problem. In this case we can modify the value of the function $q$ at each point of the object (so we can decide its shape: thus $q$ itself)  

In [3]:
# Note that the optimization is carried out only on internal points (in fact boundary poiints are conditioned)...
C_internal = C[internal_idx, internal_idx]

# Defininig J (obj function) as follows menas ~> We want to reach a designed u_target
u_target(x) = (x .- a) .* (b .- x)
u_target_num = u_target(X_internal)
J(u_num) = norm(u_num - u_target_num)^2

g(u_num) = 2 * (u_num - u_target_num)

# Define ideal parameters that leads to u_target
h_target_num = C_internal * u_target_num 
q_target_num = h_target_num + C[internal_idx, boundary_idx]*boundary_conditions
error_h(h_num) = norm(h_num-h_target_num)^2

error_h (generic function with 1 method)

Getting the sougth parameters $q$ that leads to the desired field $u_{opt}$

In [4]:
h_0 = C_internal * u_approximated
N_opt::Int64 = 1e4

h_opt_num, u_opt_num = adjoint_opt_BB(C_internal, h_0, J, g, N_opt, BB_long_step)
q_opt_num = h_opt_num + + C[internal_idx, boundary_idx]*boundary_conditions

MethodError: MethodError: Cannot `convert` an object of type Vector{Float64} to an object of type Float64

Closest candidates are:
  convert(::Type{T}, !Matched::T) where T<:Number
   @ Base number.jl:6
  convert(::Type{T}, !Matched::Number) where T<:Number
   @ Base number.jl:7
  convert(::Type{T}, !Matched::Base.TwicePrecision) where T<:Number
   @ Base twiceprecision.jl:273
  ...


## Performances

In [5]:
relative_error_on_q = norm(q_opt_num - q_target_num) / norm(q_target_num)
relative_error_on_u = norm(u_opt_num - u_target_num) / norm(u_target_num)

println("Relative error on u: $(relative_error_on_u)")
println("Relative error on q: $(relative_error_on_q)")

UndefVarError: UndefVarError: `q_opt_num` not defined

In [6]:
tmp = InfoToStep(h0, h0, h0, (h0 .- 100))
BB_long_step(tmp)

UndefVarError: UndefVarError: `h0` not defined

In [7]:
h0 = q(X[internal_idx]) - C[internal_idx, boundary_idx]*boundary_conditions
N_iter = 1e4

h_opt, u_opt = adjoint_opt_BB(C[internal_idx, internal_idx], h0, J, g, N_iter, BB_long_step, 10^(-14))
q_opt = h_opt + C[internal_idx, boundary_idx]*boundary_conditions
println("Fitness value is: $(J(u_opt))")
println("Errors on the obtained q: $(q_opt - q_target_num)")
println("Errors on the obtained u: $(u_opt - u_target_num)")
println("Relative error on u: $(norm(u_opt - u_target_num) / norm(u_target_num))")
println("Relative error on q: $(norm(q_opt - q_target_num) / norm(q_target_num))")

MethodError: MethodError: no method matching adjoint_opt_BB(::Matrix{Float64}, ::Vector{Float64}, ::typeof(J), ::typeof(g), ::Float64, ::typeof(BB_long_step), ::Float64)

Closest candidates are:
  adjoint_opt_BB(::Any, ::Any, ::Any, ::Any, !Matched::Int64, ::Any, ::Any)
   @ Main.DiscreteAdjoint ~/Uni/Tesi/thesis_code/src/DiscreteAdjoint.jl:7
  adjoint_opt_BB(::Any, ::Any, ::Any, ::Any, !Matched::Int64, ::Any)
   @ Main.DiscreteAdjoint ~/Uni/Tesi/thesis_code/src/DiscreteAdjoint.jl:7
